<a href="https://colab.research.google.com/github/skywalker0803r/Ricky/blob/master/%E6%A8%A1%E7%B5%84%E5%8C%96%E9%9B%86%E6%88%90%E5%AD%B8%E7%BF%92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# get_oof

In [0]:
from sklearn.model_selection import KFold
def get_oof(model ,x_train, y_train, x_test):
    kf = KFold(n_splits=5,random_state=0)
    oof_train = np.zeros((len(x_train),))
    oof_test = np.zeros((len(x_test),))
    oof_test_skf = np.empty((5, len(x_test)))

    for i, (train_index, test_index) in enumerate(kf.split(x_train)):
        x_tr = x_train.loc[train_index]
        y_tr = y_train.loc[train_index]
        x_te = x_train.loc[test_index]

        model.fit(x_tr, y_tr)

        oof_train[test_index] = model.predict(x_te)
        oof_test_skf[i, :] = model.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [0]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
import numpy as np
import warnings 
warnings.simplefilter('ignore')

def 集成學習(X_train,y_train,X_test): 
  
  #定義 base model
  model1 = RandomForestRegressor()
  model2 = LinearRegression()
  model3 = KNeighborsRegressor()
  
  #迴圈取得每個base model的oof_train和oof_test
  所有oof_train = np.empty((len(X_train),0))
  所有oof_test = np.empty((len(X_test),0))
  for model in [model1,model2,model3]:
    oof_train,oof_test = get_oof(model,X_train,y_train,X_test)
    所有oof_train = np.hstack((所有oof_train,oof_train))
    所有oof_test = np.hstack((所有oof_test,oof_test))
  
  #訓練 meta model
  meta_model = XGBRegressor(objective="reg:squarederror").fit(所有oof_train,y_train)
  
  #meta model 預測
  return meta_model.predict(所有oof_test)

# 評估

In [0]:
from sklearn.metrics import r2_score
import numpy as np

def rmse(y_true, y_pred):
    return np.sqrt(((y_pred - y_true) ** 2).mean())

def mape(y_true, y_pred,dropinf=False):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    ape = np.abs((y_true - y_pred) / y_true)
    if dropinf == True:
      ape = ape[np.isfinite(ape)]
    return np.mean(ape) * 100

def metric(y_true,y_pred,y_name):
  R2 = r2_score(y_true,y_pred)
  RMSE = rmse(y_true,y_pred)
  MAPE = mape(y_true,y_pred)
  return pd.DataFrame({'R2':R2,'RMSE':RMSE,'MAPE':MAPE},index=[y_name])

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# trainset

In [5]:
import pandas as pd
train = pd.read_csv('/content/drive/My Drive/台塑輕油案子/輕油實驗數據(保留大部份欄位,三個基本規則清洗過).csv')
X = train[['T10','T50','T90','N+A in Feed']].rename(columns={'N+A in Feed':'NA'})
y = train.iloc[:,17:-1]
X.join(y).head()

,T10,T50,T90,NA,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,100.5,119.2,146.5,31.978,0.272,0.132,0.059,5.293,2.570,2.819,0.494,10.395,8.070,6.411,2.917,9.138,9.649,4.810,5.373,6.405,9.759,4.590,3.661,0.875,5.257,0.525,0.319
1,99.6,117.9,145.5,31.568,0.297,0.151,0.062,5.089,2.531,2.804,0.499,10.074,7.958,6.395,2.894,8.970,9.548,4.753,5.443,6.324,9.899,4.301,2.995,0.881,5.591,1.119,0.303
2,100.0,118.8,145.6,31.344,0.262,0.126,0.061,5.107,2.571,2.771,0.492,10.069,7.913,6.378,2.890,9.006,9.591,4.778,5.468,6.360,9.983,4.274,2.979,0.865,5.641,0.964,0.289
3,100.4,118.6,142.9,31.453,0.224,0.105,0.060,4.894,2.497,2.650,0.469,10.015,7.685,6.376,2.866,9.133,9.708,4.889,5.510,6.444,10.182,4.420,2.964,0.830,5.637,0.968,0.281
4,100.4,118.1,142.2,32.190,0.243,0.117,0.061,4.946,2.503,2.695,0.477,10.053,7.765,6.394,2.877,9.101,9.676,4.855,5.500,6.416,10.115,4.347,3.725,0.835,4.823,0.969,0.290


# testset

In [6]:
test = pd.read_csv('/content/drive/My Drive/台塑輕油案子/test_without_outlier.csv')
X_test = test[['T10','T50','T90','N+A']].rename(columns={'N+A':'NA'})
y_test = test.iloc[:,4:]
y_test.columns = y.columns
X_test.join(y_test).head()

,T10,T50,T90,NA,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,98.6,119.2,146.1,40.133,1.142,0.616,0.217,3.745,2.577,4.028,0.557,7.669,5.9900,10.206,3.075,7.920,8.036,7.039,5.438,5.263,9.537,4.877,3.481,1.088,5.652,0.618,0.597
1,105.4,122.2,148.2,32.175,0.122,0.078,0.029,3.955,1.563,2.407,0.400,10.016,7.2633,6.573,2.809,9.544,9.815,5.135,5.482,6.793,10.850,4.308,3.963,1.069,6.025,0.585,0.484
2,105.8,122.7,149.2,31.428,0.098,0.064,0.020,3.920,1.447,2.296,0.396,10.236,7.3480,6.259,2.797,9.655,9.986,4.942,5.453,6.947,10.996,4.269,3.997,1.034,6.107,0.543,0.456
3,105.4,122.4,147.6,31.390,0.122,0.085,0.021,3.900,1.437,2.274,0.397,10.259,7.3560,6.234,2.820,9.670,10.146,4.928,5.476,6.953,11.026,4.264,3.982,1.013,6.054,0.545,0.449
4,105.4,122.6,148.2,32.418,0.072,0.049,0.015,3.873,1.369,2.206,0.489,10.213,7.3190,6.009,2.968,9.564,10.052,4.769,6.584,6.866,10.892,4.169,4.227,0.930,5.828,0.519,0.463


In [7]:
y_columns = y.columns.tolist()
len(y_columns)

23

# 單純建模表現

In [8]:
res_table = pd.DataFrame()
for y_name in y_columns:
  y_pred = XGBRegressor(objective="reg:squarederror").fit(X,y[y_name]).predict(X_test)
  res_table = res_table.append(metric(y_test[y_name].values,y_pred,y_name))
res_table.loc['AVG'] = res_table.mean()
res_table

,R2,RMSE,MAPE
C5NP,0.891181,0.281908,40.874745
C5IP,0.738123,0.261870,60.886170
C5N,0.930164,0.036177,39.377358
C6NP,0.127743,0.377854,7.727171
C6IP,0.871774,0.374294,11.321105
C6N,0.887861,0.554559,5.965841
C6A,-0.163214,0.184265,25.850968
C7NP,0.974167,0.353481,3.878464
C7IP,0.811875,0.479278,6.595102
C7N,0.982234,0.555457,3.397454


#集成學習表現

In [9]:
res_table = pd.DataFrame()
for y_name in y_columns:
  y_pred = 集成學習(X,y[y_name],X_test)
  res_table = res_table.append(metric(y_test[y_name].values,y_pred,y_name))
res_table.loc['AVG'] = res_table.mean()
res_table

,R2,RMSE,MAPE
C5NP,0.877850,0.298677,54.737375
C5IP,0.727706,0.267027,75.774108
C5N,0.907208,0.041701,58.037375
C6NP,0.129807,0.377406,6.831499
C6IP,0.909868,0.313809,10.060014
C6N,0.905141,0.510047,6.460065
C6A,-1.017156,0.242652,38.080851
C7NP,0.973388,0.358774,4.215875
C7IP,0.786418,0.510677,6.834151
C7N,0.967558,0.750607,4.467752
